In [2]:
#BERT-Base, Multilingual Cased
# bert-serving-start -model_dir bert/multi/multi_cased_L-12_H-768_A-12/ -num_worker=8
# https://www.kaggle.com/team-ai/shinzo-abe-japanese-prime-minister-twitter-nlp/version/1
from bert_serving.client import BertClient
import numpy as np
import pandas as pd
import os

csv_path = os.path.join('dataset', 'ja', "shinzo_abe.csv")
abe_df = pd.read_csv(csv_path)
abe_df.head()

,url,Full Name Show,Username Dir,Tweet Nav,Tweet Nav_link,Tweet Text Size Block,Tweet Text Size Link,Tweet Text Size Link_link,Profile Tweet 1,Profile Tweet 2,Profile Tweet 3,Reply,Re Tweet,Like
0,https://twitter.com/AbeShinzo,安倍晋三,AbeShinzo,Oct 21,https://twitter.com/AbeShinzo/status/921745765...,選挙期間中、自民党の候補者たちは全国各地で我々の経済政策を、安全保障を、そして日本の将来を担...,https://twitter.com/jimin_koho/status/92174405...,https://t.co/HVjFn3QJp5,"1,133 replies","5,752 retweets","14,514 likes",1.1K,5.8K,15K
1,https://twitter.com/AbeShinzo,安倍晋三,AbeShinzo,Oct 20,https://twitter.com/AbeShinzo/status/921350545...,外国人旅行者数はこの５年間で３倍になりました。自民党は高次元での観光資源施策に取り組み我が国...,pic.twitter.com/EMNDcADrYB,https://t.co/EMNDcADrYB,606 replies,"3,712 retweets","8,946 likes",606,3.7K,8.9K
2,https://twitter.com/AbeShinzo,安倍晋三,AbeShinzo,Oct 20,https://twitter.com/AbeShinzo/status/921322049...,明日１０月２１日(土) 安倍晋三総裁は岐阜県、愛知県、東京都に参ります。 詳細は是非、自民党...,https://special.jimin.jp/,https://t.co/vKXI8z5Mjr,358 replies,"2,213 retweets","4,922 likes",358,2.2K,4.9K
3,https://twitter.com/AbeShinzo,安倍晋三,AbeShinzo,Oct 19,https://twitter.com/AbeShinzo/status/920998741...,我々がスタートした返さなくても良い「給付型奨学金」や「授業料免除」といった制度を拡充し、真に...,pic.twitter.com/ZgAYDhlzDD,https://t.co/ZgAYDhlzDD,456 replies,"3,081 retweets","6,576 likes",456,3.1K,6.6K
4,https://twitter.com/AbeShinzo,安倍晋三,AbeShinzo,Oct 19,https://twitter.com/AbeShinzo/status/920991367...,明日１０月２０日(金) 安倍晋三総裁は神奈川県、東京都に参ります。 詳細は是非、自民党の特設...,https://special.jimin.jp/,https://t.co/vKXI8z5Mjr,271 replies,"2,291 retweets","5,610 likes",271,2.3K,5.6K


In [3]:
text_list = list(abe_df.loc[:, 'Tweet Text Size Block'])
text_list

['選挙期間中、自民党の候補者たちは全国各地で我々の経済政策を、安全保障を、そして日本の将来を担っていく決意と地域を支える想いを皆様に訴えかけさせていただきました。「この国を、守り抜く。」自民党はこれからも皆様とともに歩みを進めて参ります。本当に１２日間ご声援ありがとうございました。https://twitter.com/jimin_koho/status/921744056987693057 …',
 '外国人旅行者数はこの５年間で３倍になりました。自民党は高次元での観光資源施策に取り組み我が国の伝統文化財等の国内観光資源の強化を図り、さらに観光産業の活性化を実現して参ります。pic.twitter.com/EMNDcADrYB',
 '明日１０月２１日(土) 安倍晋三総裁は岐阜県、愛知県、東京都に参ります。 詳細は是非、自民党の特設サイトでご確認下さい。https://special.jimin.jp/',
 '我々がスタートした返さなくても良い「給付型奨学金」や「授業料免除」といった制度を拡充し、真に支援が必要な子供たちに高等教育の無償化を図って参ります。 自民党は誰もが学ぶ事を諦めなくて済む、そして安心して学んでいける社会を必ず創り上げます。pic.twitter.com/ZgAYDhlzDD',
 '明日１０月２０日(金) 安倍晋三総裁は神奈川県、東京都に参ります。 詳細は是非、自民党の特設サイトでご確認下さい。https://special.jimin.jp/',
 '明日１０月１９日(木) 安倍晋三総裁は奈良県、京都府、滋賀県に参ります。 詳細は是非、自民党の特設サイトでご確認下さい。https://special.jimin.jp/',
 'この３年間４０代以下で新しく農業を始めた就農者は３年連続で２万人を超えました。 自民党は若者や意欲のある農林漁業者を全力で応援し挑戦を支え、夢や希望を持てる「農政新時代」を切り拓きます。pic.twitter.com/Fp0IqNYTSk',
 '私たち自民党は日本の経済を支えている中小企業を力強く応援したい。その思いから中小・小規模事業者の固定資産税を、３年間半減する制度を始め既に３万件近い利用がありました。この制度の実現により中小企業の倒産は3割も減少しました。更に生産性を高めるための支援を大胆に進めます。p

In [6]:
bc = BertClient()
encoded = bc.encode(text_list)
encoded_df = pd.concat([pd.DataFrame(encoded), abe_df.loc[:, 'Tweet Nav']], axis=1).set_index('Tweet Nav')
encoded_df.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
Tweet Nav,,,,,,,,,,,,,,,,,,,,,
Oct 21,-1.335604,-0.409404,1.104787,-0.324539,-0.171543,0.423019,0.503926,-0.257214,-0.673734,0.154375,...,-0.110169,-1.901920,0.188825,-0.564849,1.121146,0.140741,-0.749882,0.788056,0.856203,0.512975
Oct 20,-1.158415,-0.256215,0.560778,0.205107,-0.437689,0.306825,0.304026,0.245066,-1.263636,0.062232,...,0.242830,-1.378554,-0.293925,-0.620082,0.521391,0.042936,-0.638995,1.318025,0.922051,0.724190
Oct 20,-0.842821,0.033817,0.782302,0.377611,-0.374691,-0.028288,-0.121979,0.347410,-0.298788,0.001002,...,0.224187,-1.097087,-0.761451,-0.520589,0.763951,0.316778,-0.746486,1.340120,0.774812,0.628353
Oct 19,-0.386365,-0.004206,0.269698,-0.169393,-0.375282,-0.230248,0.000604,0.366570,-0.537476,0.010343,...,-0.145124,-0.451577,-0.513285,-0.491230,0.187572,0.522614,0.264339,1.166471,0.915613,-0.149714
Oct 19,-0.810199,0.068962,0.748813,0.370197,-0.206046,0.102130,-0.142837,0.436940,-0.397143,-0.056888,...,0.416132,-1.116386,-0.936329,-0.882141,0.611757,-0.020011,-0.718369,1.490024,0.873632,0.371440


In [7]:
encoded_df.to_csv('abe_encoded.csv')